In [65]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
EPOCHS = 200

In [66]:
df = pd.read_csv("Month_value_1.csv")
print(df.head())

       Period       Revenue  Sales_quantity  Average_cost  \
0  01.01.2015  1.601007e+07         12729.0   1257.763541   
1  01.02.2015  1.580759e+07         11636.0   1358.507000   
2  01.03.2015  2.204715e+07         15922.0   1384.697024   
3  01.04.2015  1.881458e+07         15227.0   1235.606705   
4  01.05.2015  1.402148e+07          8620.0   1626.621765   

   The_average_annual_payroll_of_the_region  
0                                30024676.0  
1                                30024676.0  
2                                30024676.0  
3                                30024676.0  
4                                30024676.0  


In [67]:
values = df.iloc[:, 1:5].values
features = values.shape[1]
total_df_size = df.size
values_Nans = df.isna().sum().sum()
print(f"Total of Nans: {values_Nans / total_df_size}")

Total of Nans: 0.26666666666666666


In [68]:
"""
The total Nans of these DataSet is 26.6%
We need to fill it out
"""
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)

/var/folders/64/_d7wbs111535lmdv26qfwp000000gn/T/ipykernel_10438/646466139.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
/var/folders/64/_d7wbs111535lmdv26qfwp000000gn/T/ipykernel_10438/646466139.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


In [69]:
values = df.iloc[:, 1:5].values
features = values.shape[1]
total_df_size = df.size
values_Nans = df.isna().sum().sum()
print(f"Total of Nans after fillna: {values_Nans / total_df_size}")

Total of Nans after fillna: 0.0


In [70]:
scaler = MinMaxScaler(feature_range=(0,1))
values_evidence_normalize = scaler.fit_transform(values)

X, y = [], []
#Appending by the window_size that the 
window_size = 10
for i in range(len(values_evidence_normalize) - window_size):
    X.append(values_evidence_normalize[i: i + window_size])
    y.append(values_evidence_normalize[i + window_size])
    
X = np.array(X).astype(np.float32)
y = np.array(y).astype(np.float32)


In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.30, shuffle=False
)
print(f"X train shape: {X_train.shape}")
print(f"Y train shape: {Y_train.shape}")
print(f"X test shape: {X_test.shape}")
print(f"Y test shape: {Y_test.shape}")


X train shape: (60, 10, 4)
Y train shape: (60, 4)
X test shape: (26, 10, 4)
Y test shape: (26, 4)


In [72]:
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [73]:
class LSTM_Model(nn.Module):
    def __init__(self, output=features):
        super(LSTM_Model, self).__init__()
        
        self.LSTM_1 = nn.LSTM(input_size=X_train.shape[2],
                              hidden_size=100, 
                              num_layers=1,
                              batch_first=True)
        self.LSTM_2 = nn.LSTM(input_size=100,
                            hidden_size=75,
                            num_layers=1,
                            batch_first=True)
        self.LSTM_3 = nn.LSTM(input_size=75,
                                hidden_size=50,
                                num_layers=1,
                                batch_first=True)
        
        self.fc = nn.Linear(50, out_features=output)
    def forward(self, x):
        for lstm in [self.LSTM_1, self.LSTM_2, self.LSTM_3]:
            x, _ = lstm(x)
        return self.fc(x[:, -1, :])

In [74]:
rnn_model = LSTM_Model(features)
criterion = torch.nn.MSELoss()
optimizer = optim.AdamW(params=rnn_model.parameters(),lr=1e-4)

In [78]:
for epoch in range(EPOCHS):
    rnn_model.train()
    epoch_loss = 0.0
    
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = rnn_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    if(epoch +1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")
        

Epoch [50/200], Loss: 0.0435
Epoch [100/200], Loss: 0.0414
Epoch [150/200], Loss: 0.0393
Epoch [200/200], Loss: 0.0365


In [79]:
rnn_model.eval()
with torch.no_grad():
    y_pred = rnn_model(x_test).numpy()

mse = mean_squared_error(y_test.numpy(), y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test.numpy(), y_pred)

print("\n Results")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")


 Results
MSE: 0.0074
RMSE: 0.0858
MAE: 0.0708
